In [67]:
import torch
import torch.nn.functional as F
import torch.nn as nn
from torch_geometric.nn import NNConv
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader
from torch_geometric.nn import global_mean_pool

In [69]:
node_feature1 = torch.tensor([[1, 2, 3, 4], [2, 1, 9, 3], [1, 5, 5, 1]], dtype=torch.float)
node_feature2 = torch.tensor([[1, 2, 3, 4], [2, 1, 9, 3], [1, 5, 5, 1]], dtype=torch.float)

edge_index1 = torch.tensor([[0, 0, 1],[1, 2, 2]], dtype=torch.long)
edge_index2 = torch.tensor([[0, 0, 1],[1, 2, 2]], dtype=torch.long)

edge_feature1 = torch.tensor([[0, 0], [0, 0], [0, 0]], dtype=torch.float)
edge_feature2 = torch.tensor([[0, 0], [0, 0], [0, 0]], dtype=torch.float)

data1 = Data(x=node_feature1, edge_attr=edge_feature1, edge_index=edge_index1)
print(data1.batch)
data2 = Data(x=node_feature2, edge_attr=edge_feature2, edge_index=edge_index2)

data_list = [data1, data2]
loader = DataLoader(data_list, batch_size=2)
batch = next(iter(loader))
batch.batch

None


tensor([0, 0, 0, 1, 1, 1])

In [ ]:
node_feature = torch.tensor([[1, 2, 3, 4], [2, 1, 9, 3], [1, 5, 5, 1]], dtype=torch.float)
edge_feature = torch.tensor([[0, 1], [2, 0], [3, -2]], dtype=torch.float)
edge_index = torch.tensor([[0, 0, 1]
                           [1, 2, 2]], dtype=torch.long)
data = Data(x=node_feature, edge_attr=edge_feature, edge_index=edge_index)

In [75]:
class DGN(torch.nn.Module):
    def __init__(self):
        super(DGN, self).__init__()
        
        self.mlp1 = nn.Sequential(nn.Linear(2, 4*6), nn.ReLU())
        self.conv1 = NNConv(4, 6, self.mlp1, aggr='mean')
        
        self.mlp2 = nn.Sequential(nn.Linear(2, 6*6), nn.ReLU())
        self.conv2 = NNConv(6, 6, self.mlp2, aggr='mean')
        
        self.mlp3 = nn.Sequential(nn.Linear(2, 3*6), nn.ReLU())
        self.conv3 = NNConv(6, 3, self.mlp3, aggr='mean')
        
        
        
    def forward(self, data):
        x, edge_attr, edge_index = data.x, data.edge_attr, data.edge_index
        x = F.relu(self.conv1(x, edge_index, edge_attr))
        x = F.relu(self.conv2(x, edge_index, edge_attr))
        x = F.relu(self.conv3(x, edge_index, edge_attr))
        x = global_mean_pool(x, data.batch)
        print(x.shape)
        return x

In [76]:
dgn = DGN()

In [79]:
dgn(data2)

torch.Size([1, 3])


tensor([[0.3574, 0.2273, 0.1498]], grad_fn=<MeanBackward1>)